In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
# read data
df_ratings = pd.read_csv("file\\ml-1m\\ratings.dat", sep="::", header=None, names=["UserID", "MovieID", "Rating", "Timestamp"])
df_movies = pd.read_csv("file\\ml-1m\\movies.dat", sep="::", header=None, names=["MovieID", "Title", "Genres"])
df_users = pd.read_csv("file\\ml-1m\\users.dat", sep="::", header=None, names=["UserID", "Gender", "Age", "Occupation", "Zip-code"])

<ipython-input-3-66699cb8c506>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_ratings = pd.read_csv("file\\ml-1m\\ratings.dat", sep="::", header=None, names=["UserID", "MovieID", "Rating", "Timestamp"])
<ipython-input-3-66699cb8c506>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_movies = pd.read_csv("file\\ml-1m\\movies.dat", sep="::", header=None, names=["MovieID", "Title", "Genres"])
<ipython-input-3-66699cb8c506>:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as reg

In [4]:
# merge data
df_ratings_movies = pd.merge(df_ratings, df_movies, on='MovieID')
df = pd.merge(df_ratings_movies, df_users, on="UserID")
df.head()

,UserID,MovieID,Rating,Timestamp,Title,Genres,Gender,Age,Occupation,Zip-code
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,10,48067
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1,10,48067
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1,10,48067
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1,10,48067


In [5]:
# Delete unnecessary columns
data = df.copy()
data = data.drop(['Timestamp','Gender','Zip-code'], axis=1)
data.head()

,UserID,MovieID,Rating,Title,Genres,Age,Occupation
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama,1,10
1,1,661,3,James and the Giant Peach (1996),Animation|Children's|Musical,1,10
2,1,914,3,My Fair Lady (1964),Musical|Romance,1,10
3,1,3408,4,Erin Brockovich (2000),Drama,1,10
4,1,2355,5,"Bug's Life, A (1998)",Animation|Children's|Comedy,1,10


# Data preprocessing (leave only data with a rating of 3 or higher)

In [6]:
data_pre = data[data['Rating'] > 2]
data_pre.head()

,UserID,MovieID,Rating,Title,Genres,Age,Occupation
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama,1,10
1,1,661,3,James and the Giant Peach (1996),Animation|Children's|Musical,1,10
2,1,914,3,My Fair Lady (1964),Musical|Romance,1,10
3,1,3408,4,Erin Brockovich (2000),Drama,1,10
4,1,2355,5,"Bug's Life, A (1998)",Animation|Children's|Comedy,1,10


# Group recommender: Create a recommendation system using users' information (age, occupation)

In [36]:
# Split train, test set
x = data_pre.copy()
x_train, x_test = train_test_split(x, test_size=0.05)

In [8]:
# Recommend n popular movies by occupation
def occu_pop(dataset, occu, n):
    data_occu = dataset[dataset['Occupation'] == occu]
    data_occu = data_occu.reset_index()
    occu_pop_rank = data_occu['MovieID'].value_counts()[:n]
    recommend_movies = data_occu.loc[occu_pop_rank.index]
    return recommend_movies

# Recommend n movies depending on user's occupation
def user_occu_recommend(dataset, userid, n):
    user_occu = dataset[dataset['UserID'] == userid]['Occupation'].values[0]
    recomm = occu_pop(dataset, user_occu, n)
    return recomm
    
user_occu_recommend(x_train, 46, 5)

,index,UserID,MovieID,Rating,Title,Genres,Age,Occupation
2858,384003,5090,3052,3,Dogma (1999),Comedy,35,19
2571,488211,1835,1416,4,Evita (1996),Drama|Musical,25,19
260,896120,3807,3107,3,Backdraft (1991),Action|Drama,35,19
608,549878,1164,3822,4,"Girl on the Bridge, The (La Fille sur le Pont)...",Drama|Romance,25,19
1210,896197,3807,1101,3,Top Gun (1986),Action|Romance,35,19


In [9]:
# Recommend n popular movies by age group
def age_pop(dataset, age, n):
    data_age = dataset[dataset['Age'] == age]
    data_age = data_age.reset_index()
    age_pop_rank = data_age['MovieID'].value_counts()[:n]
    recommend_movies = data_age.loc[age_pop_rank.index]
    return recommend_movies

# Recommend n movies based on user's age
def user_age_recommend(dataset, userid, n):
    user_age = dataset[dataset['UserID'] == userid]['Age'].values[0]
    recomm = age_pop(dataset, user_age, n)
    return recomm

user_age_recommend(x_train, 46, 5)

,index,UserID,MovieID,Rating,Title,Genres,Age,Occupation
2858,983042,2326,3107,4,Backdraft (1991),Action|Drama,18,4
1196,279570,3756,941,4,"Mark of Zorro, The (1940)",Adventure,18,12
1210,722692,1778,1513,3,Romy and Michele's High School Reunion (1997),Comedy,18,4
260,533361,5705,1917,4,Armageddon (1998),Action|Adventure|Sci-Fi|Thriller,18,4
2571,246483,3391,1673,5,Boogie Nights (1997),Drama,18,4


# Performance measurements

## Popular model

In [139]:
def popular_movie(dataset, n):
    data_pop = dataset.copy()
    movie_rank = data_pop['MovieID'].value_counts()[:n]
    return movie_rank

def hitrate1(K):
    raw_ranking = popular_movie(data_pre, K)
    pred_ranking = popular_movie(x_test, K)
    return pd.DataFrame(raw_ranking.index.isin(pred_ranking.index)).value_counts(normalize=True)[True]

s = 0
for i in range(len(x_test.index)):
    s += hitrate1(100)
s /= len(x_test.index)
s

True    0.86
dtype: float64

## Genre model

In [37]:
def genre_pop(dataset, genre, n):
    dataset_genres = dataset['Genres'].str.get_dummies("|")
    select_genre = dataset_genres[dataset_genres[genre]==1]
    genre_popmovie = dataset.loc[select_genre.index]
    genre_popmovie_rank = genre_popmovie['MovieID'].value_counts()[:n]
    return genre_popmovie_rank

def user_genre(dataset, userid):
    user_data = dataset[dataset['UserID']==userid]
    user_data_genres = user_data['Genres'].str.get_dummies("|")
    user_genre_ranking = user_data_genres.sum().sort_values(ascending=False)
    return user_genre_ranking

def user_genre_recommend(dataset, userid, n):
    genre_pref = user_genre(dataset, userid)
    recomm = genre_pop(dataset, genre_pref.index[0], n)
    return recomm

def hitrate2(user, K):
    raw_recomm = user_genre_recommend(data_pre, user, K)
    pred_recomm = user_genre_recommend(x_test, user, K)
    return pd.DataFrame(raw_recomm.index.isin(pred_recomm.index)).value_counts(normalize=True)[True]

s = 0
for i in x_test['UserID'].index:
    s += hitrate2(x_test['UserID'][i], 100)
s /= len(x_test.index)
s

KeyError: True

## Group model

In [140]:
def occu_pop(dataset, occu, n):
    data_occu = dataset[dataset['Occupation'] == occu]
    occu_pop_rank = data_occu['MovieID'].value_counts()[:n]
    return occu_pop_rank

def user_occu_recommend(dataset, userid, n):
    user_occu = dataset[dataset['UserID'] == userid]['Occupation'].values[0]
    recomm = occu_pop(dataset, user_occu, n)
    return recomm

def hitrate3(user, K):
    raw_recomm = user_occu_recommend(data_pre, user, K)
    pred_recomm = user_occu_recommend(x_test, user, K)
    return pd.DataFrame(raw_recomm.index.isin(pred_recomm.index)).value_counts(normalize=True)[True]

s = 0
for i in x_test['UserID'].index:
    s += hitrate3(x_test['UserID'][i], 100)
s /= len(x_test.index)
s

True    0.579235
dtype: float64

In [141]:
def age_pop(dataset, age, n):
    data_age = dataset[dataset['Age'] == age]
    data_age = data_age.reset_index()
    age_pop_rank = data_age['MovieID'].value_counts()[:n]
    return age_pop_rank

def user_age_recommend(dataset, userid, n):
    user_age = dataset[dataset['UserID'] == userid]['Age'].values[0]
    recomm = age_pop(dataset, user_age, n)
    return recomm

def hitrate4(user, K):
    raw_recomm = user_age_recommend(data_pre, user, K)
    pred_recomm = user_age_recommend(x_train, user, K)
    return pd.DataFrame(raw_recomm.index.isin(pred_recomm.index)).value_counts(normalize=True)[True]

s = 0
for i in x_test['UserID'].index:
    s += hitrate4(x_test['UserID'][i], 100)
s /= len(x_test.index)
s

True    0.981824
dtype: float64